# Wet vegetation threshold tool

This notebook implements a widget for helping fihd good thresholds for wet vegetation.

In [20]:
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
import xarray

In [11]:


def get_mrvbf(tile_label):
    mrvbf_path_base = (
        mrvbf_base_path
    ) = "/g/data/u46/wofs/confidence_albers/MrVBF/tiles/mrvbf_{}.nc"
    mrvbf_path = mrvbf_path_base.format(tile_label.replace(",", "_"))
    da = xarray.open_dataset(mrvbf_path)
    return da

def get_tcg(tile_label):
    tif_path_base = "/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_{}_1986_2020_summary.tif"
    tif_path = tif_path_base.format(tile_label.replace(",", "_"))
    da = xarray.open_rasterio(tif_path)
    return da

In [12]:
%%bash
ls /g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/*.tif

/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_10_-15_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_10_-17_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_10_-19_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_10_-22_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_10_-32_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_12_-16_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_12_-29_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_12_-32_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_13_-27_1986_2020_summary.tif
/g/data/r78/datacube_stats/T

Unload all python modules, if you experience any issues.


In [175]:
%matplotlib widget
def show_widget(tile_label):
    tcw = get_tcw(tile_label).isel(band=3)[::5, ::5]
    tcg = get_tcg(tile_label).isel(band=1)[::5, ::5]
    
    initial_threshold = np.percentile(tcg, 95)
    tcw_mask = tcw > -600
    tcg_mask = tcg < initial_threshold
    
    output = ipywidgets.Output()
    with output:
        fig = plt.figure(figsize=(6, 7))
        gs = fig.add_gridspec(ncols=1, nrows=2, width_ratios=[1],
                              height_ratios=[1, 0.25], hspace=0.2, bottom=0.03,
                              top=0.95)
        ax = fig.add_subplot(gs[0, 0])
        im = (tcw_mask & tcg_mask).plot.imshow(interpolation='bilinear')
        hist_ax = fig.add_subplot(gs[1, 0])
        hist = hist_ax.hist(tcg.values.ravel(), bins=100)
        hist_line = hist_ax.axvline(initial_threshold, c='red')
    
    def slider_update(change):
        threshold = change['new']['value']
        thresholded = tcg < threshold
        with output:
            im.set_data(tcw_mask & thresholded)
            hist_line.set_xdata([threshold, threshold])
            im.autoscale()
            fig.canvas.draw()

    slider = ipywidgets.FloatSlider(
        value=initial_threshold,
        min=tcg.min(),
        max=tcg.max(),
        step=10,
        description='Max TCG50:',
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='.2f',
        layout=ipywidgets.Layout(width='50%', margin='0 0 0 0')
    )
    slider.observe(slider_update)

    return ipywidgets.VBox([output, slider], layout=ipywidgets.Layout(margin='0 0 0 0'))

In [176]:
show_widget('9,-31')